In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib
import glob
import os

from scipy.stats import trim_mean
from statsmodels import robust
from matplotlib import font_manager, rc
%matplotlib inline
from matplotlib import cm 
import gc  # garbage collection
from sklearn.cluster import KMeans  # k-means 군집분석
from sklearn.metrics import silhouette_score  # 군집분석 평가를 위한 silhouette 값 계산

pd.set_option('max_columns', 20, 'max_rows', 20)

%matplotlib inline

# 한글 폰트 설정|
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [4]:
base = pd.read_csv('../data/processed_data/regression.csv', encoding='cp949')
give_final = pd.read_csv('../data/processed_data/modeling.csv', encoding='cp949')

In [5]:
cluster_result = pd.read_csv('../data/processed_data/dong_cluster_result_5.csv', encoding='cp949')
cluster_result = cluster_result.rename(columns = {'emd_nm': '읍면동'})
cluster_result = cluster_result.iloc[:,:2]

In [6]:
give_final = pd.merge(give_final, cluster_result)

In [7]:
give_final.shape

(53634, 79)

In [8]:
base = pd.merge(base, cluster_result)

In [9]:
## 범주형 변수 One-hot
target = base['요일']
num = np.unique(target, axis=0)
num = num.shape[0]
encoding = np.eye(num)[target]
요일 = pd.DataFrame(encoding, columns = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']).astype('int')
base = pd.concat([base, 요일], axis=1)

target = base['주말및공휴일여부']
num = np.unique(target, axis=0)
num = num.shape[0]
encoding = np.eye(num)[target]
공휴일 = pd.DataFrame(encoding, columns = ['공휴일_X','공휴일_O']).astype('int')
base = pd.concat([base, 공휴일], axis=1)

target = base['코로나발생여부']
num = np.unique(target, axis=0)
num = num.shape[0]
encoding = np.eye(num)[target]
코로나 = pd.DataFrame(encoding, columns = ['코로나_X','코로나_O']).astype('int')
base = pd.concat([base, 코로나], axis=1)

target = base['성수기여부']
num = np.unique(target, axis=0)
num = num.shape[0]
encoding = np.eye(num)[target]
성수기 = pd.DataFrame(encoding, columns = ['성수기_X','성수기_O']).astype('int')
base = pd.concat([base, 성수기], axis=1)

target = base['월']
월 = pd.get_dummies(target)
월.columns = ['1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월']
base = pd.concat([base, 월], axis=1)
base.drop(['요일','월','주말및공휴일여부', '코로나발생여부','성수기여부'],axis=1,inplace=True)

In [10]:
non_numeric_X = ['일자','읍면동','cluster_result','월요일', '화요일', '수요일',
       '목요일', '금요일', '토요일', '일요일', '공휴일_X', '공휴일_O', '코로나_X', '코로나_O', '성수기_X',
       '성수기_O', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월',
       '11월', '12월']

In [11]:
Y = ['일별배출량합계']

In [12]:
numeric_X = list(set(set(list(base.columns)) - set(non_numeric_X) - set(Y)))

In [13]:
non_numeric_X_df = base[non_numeric_X]

In [14]:
numeric_X_df = base[numeric_X]

In [15]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [16]:
numeric_X_df = mean_norm(numeric_X_df)

In [17]:
Y_df = base[Y]

In [18]:
Y_df['일별배출량합계_log'] = Y_df['일별배출량합계'].apply(lambda x: np.log(x+1))    

/Users/jwo0201/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
Y_df = Y_df.iloc[:,1:]

In [20]:
features = pd.concat([non_numeric_X_df, numeric_X_df], axis=1)
features = pd.concat([features, Y_df], axis=1)

In [21]:
for j in range(5):
    final_df = pd.DataFrame(columns = ['변수명','R_2', 'ad_R_2', 'P_value', 'coef'])
    df = features[features['cluster_result'] == j]
    df['intercept'] = 1
    
    ## 수치형 변수
    for i in list(features.columns)[3:-1]:
        lm = sm.OLS(df['일별배출량합계_log'], df[['intercept', i]])
        results = lm.fit()
        new_df = pd.DataFrame(columns = ['변수명', 'R_2', 'ad_R_2', 'P_value', 'coef'])
        new_df['변수명'] = [i]
        new_df['R_2'] = [results.rsquared]
        new_df['ad_R_2'] = [results.rsquared_adj]
        new_df['P_value'] = [results.pvalues[i]]
        new_df['coef'] = [results.params[i]]
        final_df = pd.concat([final_df, new_df])
    
    final_df.to_csv('../data/processed_data/cluster_{}_features_단순선형회귀.csv'.format(j), encoding='cp949', index=False)

/Users/jwo0201/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/jwo0201/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/jwo0201/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [46]:
a = list(pd.read_csv('../data/processed_data/cluster_0_features_단순선형회귀.csv', encoding='cp949').query('P_value <= 0.05 & R_2 >= 0.3').sort_values(by=['R_2','coef'], ascending=False).변수명.values)
b = list(pd.read_csv('../data/processed_data/cluster_1_features_단순선형회귀.csv', encoding='cp949').query('P_value <= 0.05 & R_2 >= 0.3').sort_values(by=['R_2','coef'], ascending=False).변수명.values)
c = list(pd.read_csv('../data/processed_data/cluster_2_features_단순선형회귀.csv', encoding='cp949').query('P_value <= 0.05 & R_2 >= 0.3').sort_values(by=['R_2','coef'], ascending=False).변수명.values)
d = list(pd.read_csv('../data/processed_data/cluster_3_features_단순선형회귀.csv', encoding='cp949').query('P_value <= 0.05 & R_2 >= 0.3').sort_values(by=['R_2','coef'], ascending=False).변수명.values)
e = list(pd.read_csv('../data/processed_data/cluster_4_features_단순선형회귀.csv', encoding='cp949').query('P_value <= 0.05 & R_2 >= 0.3').sort_values(by=['R_2','coef'], ascending=False).변수명.values)

cluster_list = [a,b,c,d,e]

duplicates = set(a) & set(b) & set(c) & set(d) & set(e)
print('중복된 변수', ':', list(duplicates))
print('                      ')
print('                      ')

num = 0
for i in cluster_list:
    print('----',num,'클러스터 중복 X 변수','----')
    print(list(set(i) - duplicates))
    print('                      ')
    print('                      ')
    num += 1

중복된 변수 : ['일별외식소비합계', '일배출횟수합계', '남_내거주', '남_총거주', '소비액합계', '여_총거주', '일배출금액합계', '여_내거주', '남녀_총거주', '일지출횟수합계', '총_내거주', '여성_거주평균', '일별내식소비합계']
                      
                      
---- 0 클러스터 중복 X 변수 ----
['저녁_거주인구', '노인거주인구', '남성_근무평균', '아침_거주인구', '저녁_근무인구', '저녁_방문인구', '남성_거주평균', '점심_거주인구']
                      
                      
---- 1 클러스터 중복 X 변수 ----
['저녁_거주인구', '총_외거주', '아침_장외_거주인구', '노인거주인구', '남_외거주', '아침_거주인구', '여_외거주', '회당배출금액', '회당배출량', '남성_거주평균', '점심_거주인구']
                      
                      
---- 2 클러스터 중복 X 변수 ----
['저녁_거주인구', '여성_근무평균', '노인거주인구', '남성_방문평균', '여성_방문평균', '아침_방문인구', '남성_근무평균', '아침_거주인구', '회당배출금액', '회당배출량', '저녁_근무인구', '아침_근무인구', '저녁_방문인구', '남성_거주평균', '점심_방문인구', '점심_거주인구', '점심_근무인구']
                      
                      
---- 3 클러스터 중복 X 변수 ----
['아침_장외_거주인구', '여성_근무평균', '노인거주인구', '장외_거주평균', '남성_방문평균', '남성_근무평균', '저녁_근무인구', '점심_장외_근무인구', '남성_거주평균', '점심_거주인구', '저녁_거주인구', '저녁_장외_거주인구', '여_외거주', '아침_거주인구', '회당배출금액', '점심_근무인구', '총_외거

In [47]:
chuchul = ['일자', '읍면동', '월', '요일', '코로나발생여부', '주말및공휴일여부', '성수기여부']

In [48]:
aaa_0 = chuchul + a  + Y
aaa_1 = chuchul + b  + Y
aaa_2 = chuchul + c  + Y
aaa_3 = chuchul + d  + Y
aaa_4 = chuchul + e  + Y

In [49]:
print('---------------------')
ttt = give_final[give_final.cluster_result == 0]
ttt_final = ttt[aaa_0].reset_index(drop=True).sort_values(by=['일자','읍면동']).reset_index(drop=True)
ttt_final.to_csv('../data/processed_data/클러스터_0_최종피쳐.csv', index=False, encoding='cp949')
print(ttt_final.shape)
print(ttt_final.shape[0]/1277)

ttt = give_final[give_final.cluster_result == 1]
ttt_final = ttt[aaa_1].reset_index(drop=True).sort_values(by=['일자','읍면동']).reset_index(drop=True)
ttt_final.to_csv('../data/processed_data/클러스터_1_최종피쳐.csv', index=False, encoding='cp949')
print(ttt_final.shape)
print(ttt_final.shape[0]/1277)

ttt = give_final[give_final.cluster_result == 2]
ttt_final = ttt[aaa_2].reset_index(drop=True).sort_values(by=['일자','읍면동']).reset_index(drop=True)
ttt_final.to_csv('../data/processed_data/클러스터_2_최종피쳐.csv', index=False, encoding='cp949')
print(ttt_final.shape)
print(ttt_final.shape[0]/1277)

ttt = give_final[give_final.cluster_result == 3]
ttt_final = ttt[aaa_3].reset_index(drop=True).sort_values(by=['일자','읍면동']).reset_index(drop=True)
ttt_final.to_csv('../data/processed_data/클러스터_3_최종피쳐.csv', index=False, encoding='cp949')
print(ttt_final.shape)
print(ttt_final.shape[0]/1277)

ttt = give_final[give_final.cluster_result == 4]
ttt_final = ttt[aaa_4].reset_index(drop=True).sort_values(by=['일자','읍면동']).reset_index(drop=True)
ttt_final.to_csv('../data/processed_data/클러스터_4_최종피쳐.csv', index=False, encoding='cp949')
print(ttt_final.shape)
print(ttt_final.shape[0]/1277)


---------------------
(16601, 29)
13.0
(5108, 32)
4.0
(14047, 38)
11.0
(7662, 44)
6.0
(10216, 26)
8.0
